In [1]:
import subprocess
import pathlib

# from scripts.helpers.source_code_helpers import find_py_files
# from helpers.source_code_helpers import find_py_files
from pyphocorehelpers.Filesystem.source_code_helpers import find_py_files
from pyphocorehelpers.Filesystem.generate_code_index import build_code_index
# from generate_code_index import build_code_index

In [2]:
root_project_path = pathlib.Path("C:/Users/pho/repos/Spike3DWorkEnv/")
cscope_files = root_project_path / "cscope.files"
cscope_out = root_project_path / "cscope.out"
tags = root_project_path / "tags"
db_path = root_project_path / "myproject.db"

repos_source_paths = ["C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy", "C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src", "C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src"]
repos_exclude_dir_lists = [None, None, ["pyphoplacecellanalysis/External"]]

all_found_files = []
for a_src_path, exclude_dirs in zip(repos_source_paths, repos_exclude_dir_lists):
    all_found_files.extend(find_py_files(a_src_path, exclude_dirs = exclude_dirs))

# all_found_files

# find_py_files("C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy")
# find_py_files("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src")
# find_py_files("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src", exclude_dirs = ["pyphoplacecellanalysis/External"])



In [3]:
# Write the file paths to cscope.files
with open(cscope_files, "w") as f:
	print(f'writing to {cscope_files}...')
	for file_path in all_found_files:
		f.write(str(file_path) + "\n")
	print('\tdone.')

writing to C:\Users\pho\repos\Spike3DWorkEnv\cscope.files...
	done.


# Full Implementation


In [ ]:

def build_code_index(project_path, exclude_dirs=[]):
    project_path = pathlib.Path(project_path)
    cscope_files = project_path / "cscope.files"
    cscope_out = project_path / "cscope.out"
    tags = project_path / "tags"
    db_path = project_path / "myproject.db"

    # Find all .py files in the project directory and its subdirectories
    py_files = project_path.glob("**/*.py")
    py_files = [file_path for file_path in py_files] # to list

    excluded_py_files = []
    if exclude_dirs is not None:
        # Find all .py files in the project directory and its subdirectories, excluding the 'my_exclude_dir' directory
        exclude_paths = [project_path.joinpath(a_dir) for a_dir in exclude_dirs]
        for an_exclude_path in exclude_paths:
            excluded_py_files.extend([file_path for file_path in an_exclude_path.glob("**/*.py")])

    included_py_files = [x for x in py_files if x not in excluded_py_files]

    # Write the file paths to cscope.files
    with open(project_path / "cscope.files", "w") as f:
        for file_path in included_py_files:
            f.write(str(file_path) + "\n")

    # Generate the cscope index
    subprocess.run(["pycscope", "-i", str(cscope_files)], cwd=str(project_path), check=True)
    
    # Generate the ctags index
    subprocess.run(["ctags", "--fields=+i", "-n", "-L", str(cscope_files)], cwd=str(project_path), check=True)
    
    # Generate the CodeQL database
    subprocess.run(["cqmakedb", "-s", str(db_path), "-c", str(cscope_out), "-t", str(tags), "-p"], cwd=str(project_path), check=True)



In [4]:
build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy")

In [5]:
build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src")

In [6]:
build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src", exclude_dirs = ["pyphoplacecellanalysis/External"])

CalledProcessError: Command '['cqmakedb', '-s', 'C:\\Users\\pho\\repos\\Spike3DWorkEnv\\pyPhoPlaceCellAnalysis\\src\\myproject.db', '-c', 'C:\\Users\\pho\\repos\\Spike3DWorkEnv\\pyPhoPlaceCellAnalysis\\src\\cscope.out', '-t', 'C:\\Users\\pho\\repos\\Spike3DWorkEnv\\pyPhoPlaceCellAnalysis\\src\\tags', '-p']' returned non-zero exit status 1.

In [ ]:
'src'
'src'


"C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src", exclude_dirs = ["pyphoplacecellanalysis/External"]